In [327]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
from datetime import datetime
import time
from datetime import timedelta
import re
import warnings
warnings.filterwarnings('ignore')

In [328]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

def preprocess_input_data(data):
    df = pd.DataFrame(data)
    if 'Delhi' in df['Destination'].values:
        df['Destination'] = df['Destination'].replace('Delhi', 'New Delhi')
        
    df.dropna(subset=['Route'], inplace=True)
    
    df.dropna(subset=['Total_Stops'], inplace=True)
    df.drop_duplicates(inplace=True)
    Q1=df['Price'].quantile(0.25)
    Q3=df['Price'].quantile(0.75)
    IQR=Q3-Q1
    lower_limit=Q1-1.5*IQR
    upper_limit=Q3+1.5*IQR
    if ((len(df.loc[df['Price'] > upper_limit]) / len(df)) <= 0.5) or ((len(df.loc[df['Price'] < lower_limit]) / len(df)) <= 0.5):
        df.loc[df['Price'] > upper_limit, "Price"] = df["Price"].median()
        df.loc[df['Price'] < lower_limit, "Price"] = df["Price"].median()
    label = LabelEncoder()
    df.Airline=label.fit_transform(df.Airline)
    df.Source=label.fit_transform(df.Source)    
    df.Destination=label.fit_transform(df.Destination)    
    df.replace({'non-stop':0,'1 stop':1,'2 stops':2,
            '3 stops':3,'4 stops':4},inplace=True)
    df["Date_of_Journey"]=pd.to_datetime(df["Date_of_Journey"],format="%d/%m/%Y")
    df['Journey_Date'] = df['Date_of_Journey'].dt.day
    df['Journey_Month'] = df['Date_of_Journey'].dt.month
    df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
    df['Dep_Minute'] = pd.to_datetime(df['Dep_Time']).dt.minute
    df['Arr_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
    df['Arr_Minute'] = pd.to_datetime(df['Arrival_Time']).dt.minute
    df['Duration_hours'] = df['Duration'].str.extract(r'(\d+)h').astype(float)
    df['Duration_minutes'] = df['Duration'].str.extract(r'(\d+)min').astype(float)
    df['Duration_hours'].fillna(0, inplace=True)
    df['Duration_minutes'].fillna(0, inplace=True)
    df.drop(["Route"],axis=1,inplace=True)
    df.drop(["Additional_Info"],axis=1,inplace=True)
    df.drop(columns=['Date_of_Journey'],inplace=True)
    df.drop(columns=['Dep_Time'],inplace=True)
    df.drop(columns=['Arrival_Time'],inplace=True)
    df.drop(columns=['Duration'],inplace=True)
    df.drop(columns=['Duration_minutes'],inplace=True)
    x=df.drop("Price",axis=1)
    scaler=MinMaxScaler()
    x=scaler.fit_transform(x)
    df.columns
    return df

In [329]:
# Loading the dataset
data = pd.read_excel("Flight_Fare.xlsx")
# converting to pandas dataframe
df = pd.DataFrame(data)

In [330]:
# Replace 'Delhi' with 'New Delhi' in the 'Destination' column
# as we need to clean the data before analysing it
df['Destination'] = df['Destination'].replace('Delhi', 'New Delhi')

In [331]:
# verifying if all the columns mentioned above are dropped
print(df.columns)

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')


In [332]:
# checking the missing values present in the data
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [333]:
# dropping missing values from route and total_stops columns
df.dropna(subset=['Route'], inplace=True)
df.dropna(subset=['Total_Stops'], inplace=True)

In [334]:
# Re check for missing values
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

### Check for duplicates and handling them

In [335]:
df.duplicated().sum()
#Remove Duplicate Rows
df.drop_duplicates(inplace=True)
# Re check for duplicate values
df.duplicated().sum()

0

### Check for outliers 

In [336]:
# find Q1 and Q3
Q1=df['Price'].quantile(0.25)
Q3=df['Price'].quantile(0.75)
# find IQR
IQR=Q3-Q1
# find lower limit and upper limit
lower_limit=Q1-1.5*IQR
upper_limit=Q3+1.5*IQR

In [337]:
if ((len(df.loc[df['Price'] > upper_limit]) / len(df)) <= 0.5) or ((len(df.loc[df['Price'] < lower_limit]) / len(df)) <= 0.5):
    df.loc[df['Price'] > upper_limit, "Price"] = df["Price"].median()
    df.loc[df['Price'] < lower_limit, "Price"] = df["Price"].median()

In [338]:
## Encoding Airline, Source and Destination using Label Encoder

from sklearn.preprocessing import LabelEncoder
#importing label encoder from sklearn 

label = LabelEncoder()#object creation

df.Airline=label.fit_transform(df.Airline)
#applying label encoder 

In [339]:
df.Source=label.fit_transform(df.Source)

In [340]:
df.Destination=label.fit_transform(df.Destination)

In [341]:
## Manual encoding Total_Stops
df.replace({'non-stop':0,'1 stop':1,'2 stops':2,
            '3 stops':3,'4 stops':4},inplace=True)

In [342]:
# Convert date_of_journey column to datetime object
df["Date_of_Journey"]=pd.to_datetime(df["Date_of_Journey"],format="%d/%m/%Y")

# Extract date and month from date_of_journey
df['Journey_Date'] = df['Date_of_Journey'].dt.day
df['Journey_Month'] = df['Date_of_Journey'].dt.month

# Extract hours and minutes from dep_time
df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_Minute'] = pd.to_datetime(df['Dep_Time']).dt.minute

# Extract hours and minutes from arrival_time
df['Arr_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arr_Minute'] = pd.to_datetime(df['Arrival_Time']).dt.minute

# extract hours and minutes from Duration
df['Duration_hours'] = df['Duration'].str.extract(r'(\d+)h').astype(float)
df['Duration_minutes'] = df['Duration'].str.extract(r'(\d+)min').astype(float)
df['Duration_hours'].fillna(0, inplace=True)
df['Duration_minutes'].fillna(0, inplace=True)



In [343]:
df.drop(["Route"],axis=1,inplace=True)
df.drop(["Additional_Info"],axis=1,inplace=True)
df.drop(columns=['Date_of_Journey'],inplace=True)
df.drop(columns=['Dep_Time'],inplace=True)
df.drop(columns=['Arrival_Time'],inplace=True)
df.drop(columns=['Duration'],inplace=True)
df.drop(columns=['Duration_minutes'],inplace=True)
x=df.drop("Price",axis=1)

In [344]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
x=scaler.fit_transform(x)


## 6. Model Creation

In [345]:
# separating predictors and target 
x=df.drop(["Price"],axis=1)
y=df["Price"]

In [346]:
# separating training and testing data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [347]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
import math
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=100,max_depth=15,
                            min_samples_split=5,min_samples_leaf=5,
                            max_features=0.8,bootstrap=True)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
mse3=mean_squared_error(y_test,y_pred)
mae3=mean_absolute_error(y_test,y_pred)
model_score=r2_score(y_test,y_pred)
model_score

0.8352160757358345

In [348]:
import joblib
joblib.dump(model, 'flightPredictionModel.pkl')

['flightPredictionModel.pkl']

In [349]:
flightPredictionloaded_model = joblib.load('your_model_filename.pkl')


In [350]:
input_data = pd.read_excel("Flight_Fare2.xlsx")
input_data = preprocess_input_data(input_data)
true_values = input_data['Price']  # 'Price' is the column with true values

input_data.drop('Price', axis=1, inplace=True)
predictions = flightPredictionloaded_model.predict(input_data)
r2 = r2_score(true_values, predictions)
r2


0.8411554818699649